In [1]:
import os
import requests
import shutil
import pandas as pd
import time
import random as rd

In [2]:
import re

In [3]:
def read_file(filepath, sheet, num_rows=None):
    """read the excel file into a dataframe"""
    df = pd.read_excel(filepath, sheet_name=sheet, nrows=num_rows)
    return df

In [4]:
def clean_df(df):
    """rename columns, drop faulty rows and reset the index to job_codes"""
    # clean up the column names
    df.columns = ['job_code', 'functional_area', 'job_description', 
                  'remark', 'job_title', 'start_date', 'expiry_date']
    
    #dropping faulty job_codes
    index_names = df[df['job_code'].map(type) != int].index
    df.drop(index_names, inplace=True)
    
    #reset the index to the job code
    df.set_index('job_code', inplace=True)
    
    #sort by job codes
    df.sort_index(axis=0, inplace=True)
    
    return df

In [5]:
def find_between(s, first, last):
    """extract the string between two patterns"""
    try:
        start = s.index(first) + len(first)
        end = s.index(last, start)
        return s[start:end]
    except ValueError:
        return ""

In [6]:
def create_url(df):
    """extract the string containing the image and generate the image"""
    
    #extract the image text
    df['image_string'] = df['remark'].apply(lambda x: find_between(str(x), "Flogo%2F", "%22"))
    
    df['image_source'] = df['image_string'].apply(lambda x:str(x).split('%2F')[0] if '%2F' in x else x)
    
    df['image_code'] = df['image_string'].apply(lambda x:str(x).split('%2F')[1].split('.')[0] if '%2F' in x else x)
    
    # generate the relevant url
    df['image_url'] = df['image_string'].apply(lambda x:"http://topjobs.lk/logo/{}/{}".format(str(x).split('%2F')[0], str(x).split('%2F')[1]) if '%2F' in x else x)
                             
    df['image_url'] = df['image_url'].apply(lambda x: re.sub(r"\+", "%20", str(x)))

In [7]:
def get_images(df):
    """takes the dataframe as the input, uses the url column to download the image, and saves it locally"""
    
    file1 = open("E:/future_of_work/sample_images_8th_sep/os_error_codes_8th.txt","a+")
    
    file2 = open("E:/future_of_work/sample_images_8th_sep/no_url_codes_8th.txt","a+")
    
    for job_code in df.index[110000:]:
        if '%2F' in df.loc[job_code, 'image_string']:
            if '+' not in df.loc[job_code, 'image_url']:
                response = requests.get(df.loc[job_code, 'image_url'], stream=True)
                image_file = open("E:/future_of_work/sample_images_8th_sep/{}.{}".format(str(job_code), df.loc[job_code, 'image_url'].split('.')[-1]), 'wb')
                response.raw.decode_content = True
                shutil.copyfileobj(response.raw, image_file)
                image_file.close()
            else: 
                file1.write(str(job_code) + "\n")
        else:
            file2.write(str(job_code) + "\n")
            
        time.sleep(rd.uniform(0.2, 0.4))
        
    file1.close()

In [8]:
location = "E:/future_of_work/data/full_set_excel_sheet/20200804-topjobs-2019-20-vacancy-data.xls"
sheet1 = "2020-01-01 to 2020-06-30"

# read the file
df_tj = read_file(location, sheet1)

# clean the dataframe
df_tj = clean_df(df_tj)

# generate the urls
create_url(df_tj)


In [9]:
df_tj.shape

(38484, 10)

In [13]:
s = df_tj['image_source'].value_counts()

In [18]:
s.sum()

38484

In [22]:
100*s[s.values>50].sum()/s.sum()

94.7276790354433

### Make that Greater than value an argument to a function - so that you can control it

In [15]:
s50 = s[s>50]
s

DEFZZZ        14845
0000000375     5640
0000000403     2258
               2231
0000000023     1624
              ...  
0000000489        1
0000000183        1
0000000176        1
0000000037        1
0000000217        1
Name: image_source, Length: 163, dtype: int64

In [ ]:
a = "3catn"

In [ ]:
df_tj[df_tj['image_code'] == a]

In [ ]:
df_tj[df_tj['image_source'] == "0000000062"]['image_url'].iloc[1]

In [ ]:
df_tj['remark']

In [ ]:
def main():
    location = "E:/future_of_work/data/full_set_excel_sheet/20200804-topjobs-2019-20-vacancy-data.xls"
    sheet1 = "2020-01-01 to 2020-06-30"
    
    # read the file
    df_tj = read_file(location, sheet1, num_rows=100)
    
    # clean the dataframe
    df_tj = clean_df(df_tj)
    
    # generate the urls
    create_url(df_tj)
    
    # download the images
    get_images(df_tj)

In [ ]:
if __name__ == '__main__':
    main()

In [ ]:
df_tj.shape

#### Steps of extraction

1. 

In [23]:
p1 = "9087cAssociate+Software+Engineer"
p2 = "5538cAssociate+Service+Desk+Analyst+"

In [ ]:
import re

# Define a string
orgStr = "It is a rainy day in a rainy week"

# Replace the string
repStr = re.sub("rainy", "sunny", orgStr)

# Print the original string
print("Original Text:", orgStr)

# Print the replaced string
print("Replaced Text:", repStr)

In [24]:
print(re.sub(r"\+", "%20", p1))
print(re.sub(r"\+", "%20", p2))

9087cAssociate%20Software%20Engineer
5538cAssociate%20Service%20Desk%20Analyst%20
